<a href="https://colab.research.google.com/github/chohyungrae/Machine-Learning-Deep-Learning-Code-Learning/blob/master/Softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참조사이트: https://yamalab.tistory.com/87

softmax함수 수식
![대체 텍스트](https://miro.medium.com/max/1400/0*tGSrq3hfKFBgKntB)
일반적인 Neural Network의 activation function의 차이점 


*   일반적:유닛 k의 출력 Zk는 입력 Uk로부터만 결정되는 것
*   소프트맥수 함수;유닛 k의 출력 Zk는 Sum(Uk)로 결정된다.
---
시그모이드 함수는 로지스틱 함수의 한 케이스라 볼 수 있는데
인풋의 개수에 따라 

- input값이 하나밖에 없다면? sigmoid함수(logic함수)
- input값이 여러개면? softmax함수



![대체 텍스트](https://miro.medium.com/max/1000/0*pK080h7eb0348DBm)
j는 1부터 K까지의 범위를 가지고, 

z는 K차원 벡터

예) 작은 숫자, 3과 5가 있다.

숫자가 두 개이기 때문에, K는 2가 되고, z는 2차원벡터, 

K=[3, 5]가 된다.

=> somftmax식에 대입함으로써, 두 숫자를 확률처럼 만들 수 있다.

![대체 텍스트](https://t1.daumcdn.net/cfile/tistory/9919053B5B41E19529)


*   Softmax는 정답 클래스를 one-hot encoding의 방법으로 학습시킨다.
*   숫자(0.7, 0.2, 0.1)이 softmax를 통과하면 정답 데이터인 L은 확률값으로 바뀐다. 



In [1]:
import numpy as np


'''

- GD를 이용하여 Softmax regression 학습
- 참고 : https://deepnotes.io/softmax-crossentropy

'''

class SoftmaxRegression:
    def __init__(self, learning_rate=0.01, threshold=0.01, max_iterations=100000, verbose=False, reg_strength=1e-5):
        self._learning_rate = learning_rate  # 학습 계수
        self._max_iterations = max_iterations  # 반복 횟수
        self._threshold = threshold  # 학습 중단 계수
        self._verbose = verbose  # 중간 진행사항 출력 여부
        self._reg_strength = reg_strength # 정규화 파라미터 계수

    # theta(W) 계수들 return
    def get_coeff(self):
        return self._W

    # softmax function
    def softmax_func(self, x_data):
        predictions = x_data - (x_data.max(axis=1).reshape([-1, 1]))
        softmax = np.exp(predictions)
        softmax /= softmax.sum(axis=1).reshape([-1, 1])
        return softmax

        # prediction result example
        # [[0.01821127 0.24519181 0.73659691]
        # [0.87279747 0.0791784  0.04802413]
        # [0.05280815 0.86841135 0.0787805 ]]

    # cost function 정의
    def cost_func(self, softmax, y_data):
        sample_size = y_data.shape[0]

        # softmax[np.arange(len(softmax)), np.argmax(y_data, axis=1)]
        # --> 해당 one-hot 의 class index * 해당 유닛의 출력을 각 row(1개의 input row)에 대해 계산
        # --> (n, 1) 의 shape
        cost = -np.log(softmax[np.arange(len(softmax)), np.argmax(y_data, axis=1)]).sum() 
        cost /= sample_size
        cost += (self._reg_strength * (self._W**2).sum()) / 2
        return cost

    # gradient 계산 (regularized)
    def gradient_func(self, softmax, x_data, y_data):
        sample_size = y.shape[0]

        # softmax cost function의 미분 결과는 pi−yi 이므로,
        # softmax가 계산된 matrix에서, (해당 one-hot 의 class index * 해당 유닛)에 해당하는 유닛 위치에 -1을 더해줌.
        softmax[np.arange(len(softmax)), np.argmax(y_data, axis=1)] -= 1
        gradient = np.dot(x_data.transpose(), softmax) / sample_size
        gradient += self._reg_strength * self._W
        return gradient

    # learning
    def fit(self, x_data, y_data):
        num_examples, num_features = np.shape(x_data)
        num_classes = y.shape[1]

        # 가중계수 초기화
        self._W = np.random.randn(num_features, num_classes) / np.sqrt(num_features / 2)

        for i in range(self._max_iterations):
            
            # y^ 계산
            z = np.dot(x_data, self._W)
            softmax = self.softmax_func(z)

            # cost 함수
            cost = self.cost_func(softmax, y_data)

            # softmax 함수의 gradient (regularized)
            gradient = self.gradient_func(softmax, x_data, y_data)

            # gradient에 따라 theta 업데이트
            self._W -= self._learning_rate * gradient

            # 판정 임계값에 다다르면 학습 중단
            if cost < self._threshold:
                return False

            # 100 iter 마다 cost 출력
            if (self._verbose == True and i % 100 == 0):
                print ("Iter(Epoch): %s, Loss: %s" % (i, cost))

    # prediction
    def predict(self, x_data):
        return np.argmax(x_data.dot(self._W), 1)